In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

# 데이터 구성 확인 및 전처리

In [24]:
# 필요 라이브러리 호출
import dotenv
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import warnings

In [ ]:
df_2201 = pd.read_csv('data/서울특별시 공공자전거 대여이력 정보_2201.csv', encoding='cp949')

In [ ]:
df_2202 = pd.read_csv('data/서울특별시 공공자전거 대여이력 정보_2202.csv', encoding='cp949')
df_2202.columns == df_2201.columns # 같은 년도의 데이터 컬럼명 일치

In [ ]:
df_2202.columns

In [ ]:
df_2301 = pd.read_csv('data/서울특별시 공공자전거 대여이력 정보_2301.csv', encoding='cp949')
df_2301.columns # 23년 데이터가 더 세분화

In [16]:
df_2401 = pd.read_csv('data/서울특별시 공공자전거 대여이력 정보_2401.csv', encoding='cp949')

In [ ]:
df_2401.columns # 24년 데이터가 더 세분화

## 연도별 데이터 컬럼의 공통부분들만 모아서 새로운 데이터로 전처리가 필요
- 22년 ~ 24년의 데이터 : 월별 시간대별 사용량을 체크할 수 있는 데이터셋
- 그러나 현재 columns의 구성요소 차이가 발생하는 중으로 공통부분들만 모아서 새로운 데이터로 처리

In [ ]:
# file_path = f'data/서울특별시 공공자전거 대여이력 정보_{year_month}.csv'
df_total = pd.DataFrame(columns=['대여일시','대여 대여소번호'])
df_total.head(2)

In [ ]:
Years = [22,23,24] # 3개년 데이터 활용
Months = [i for i in range(1,13)] # 12개월치
for year in Years :
    for month in Months :
        year_month = str(year) + (str(month) if len(str(month)) == 2 else '0'+str(month))
        file_path = f'data/서울특별시 공공자전거 대여이력 정보_{year_month}.csv'
        df_temp = pd.read_csv(file_path, encoding='cp949').iloc[:,:3]
        df_total = pd.concat((df_total,df_temp[['대여일시','대여 대여소번호']]))
        print(year_month, '진행 완료')
    print('다음해에 적용합니다')

In [ ]:
df_total

In [10]:
df_total.reset_index(drop=True,inplace=True)

In [11]:
# 대여소 번호 체크
stop_info = pd.read_csv('data/공공자전거대여소정보.csv').iloc[:,:3]
stop_info.dropna(inplace=True)

In [12]:
stop_info['대여소번호'] = stop_info['대여소번호'].astype(int,copy = False)

In [ ]:
stop_info

In [14]:
# df_total에 stop_info 대여소 번호에 따라 대여소 이름과 행정구 붙이기

In [15]:
df_total.iloc[10000000]
# 일반시민이 이용하는 대여소 번호만 추출

자전거번호                 SPB-53237
대여일시        2022-05-30 22:05:04
대여 대여소번호                 3573.0
Name: 10000000, dtype: object

In [ ]:
min(stop_info['대여소번호'].tolist()),max(stop_info['대여소번호'].tolist()) # (102, 6178)

In [20]:
import numpy as np
# df_total['대여 대여소번호'] = df_total['대여 대여소번호'].astype(str).replace('\\N',np.nan) # 18000여개의 누락, 그러나 고유네임으로 사용하는 대여소 번호를 대체하는 것은 불가하다고 판단, drop 처리
# df_total.dropna(inplace=True)
df_total['대여 대여소번호'] = df_total['대여 대여소번호'].astype(float)

In [21]:
df_total

,자전거번호,대여일시,대여 대여소번호
2,SPB-33633,2022-01-02 16:02:42,3.0
3,SPB-49457,2022-01-06 08:09:11,3.0
4,SPB-32344,2022-01-11 17:53:40,3.0
5,SPB-50724,2022-01-17 16:02:21,3.0
6,SPB-34117,2022-01-17 17:57:28,3.0
...,...,...,...
132528107,SPB-41249,2024-12-31 23:14:10,4032.0
132528108,SPB-48042,2024-12-31 21:45:25,3759.0
132528109,SPB-66936,2024-12-31 23:46:21,4629.0
132528110,SPB-50325,2024-12-31 23:46:26,4629.0


In [22]:
df_total = df_total[(df_total['대여 대여소번호']>=102)&(df_total['대여 대여소번호']<=6178)]

In [25]:
df_total = df_total.iloc[:,1:]

In [26]:
def data_plus(row):
    row = row.copy()
    try:
        match = stop_info[stop_info['대여소번호'].astype(str) == str(row['대여 대여소번호'])].iloc[0]
        row['대여소이름'] = match['대여소이름']
        row['행정구'] = match['행정구']
    except IndexError:
        row['대여소이름'] = np.nan
        row['행정구'] = np.nan
    finally :
        print('작업진행중입니다')
    return row

In [27]:
data_plus(df_total.iloc[0])

작업진행중입니다


대여일시        2022-01-01 09:02:01
대여 대여소번호                  102.0
대여소이름                       NaN
행정구                         NaN
Name: 21, dtype: object

In [ ]:
df_total.apply(data_plus, axis=1) # 시간이 너무 오래 걸림(O(n)이라고 해도 단위가 너무큼. merge 방식 전환)

In [36]:
# stop_info
# df_total['대여소번호'] = df_total['대여 대여소번호']
df_total.drop('대여 대여소번호', axis=1, inplace=True)

In [38]:
df_final = df_total.merge(stop_info, how='left', on='대여소번호')

In [ ]:
df_final.isna().sum() # 3563067개 대여소번호와 대여소이름이 맞지 않음

대여일시           0
대여소번호          0
대여소이름    3563067
행정구      3563067
dtype: int64

In [ ]:
3563067/df_final.shape[0] # 약 2.7%에 해당하며, 이는 일반 시민이 사용하는 것이 아니라 정비센터 등의 목적으로 사용하는 대여소

0.026923152920746356

In [45]:
df_final.dropna(inplace=True)

In [46]:
df_final

,대여일시,대여소번호,대여소이름,행정구
0,2022-01-01 09:02:01,102.0,망원역 1번출구 앞,마포구
1,2022-01-01 09:42:06,102.0,망원역 1번출구 앞,마포구
2,2022-01-01 10:05:30,102.0,망원역 1번출구 앞,마포구
3,2022-01-01 11:14:51,102.0,망원역 1번출구 앞,마포구
4,2022-01-01 12:22:38,102.0,망원역 1번출구 앞,마포구
...,...,...,...,...
132342110,2024-12-31 23:14:10,4032.0,불암주유소 앞,노원구
132342111,2024-12-31 21:45:25,3759.0,이대서울병원,강서구
132342112,2024-12-31 23:46:21,4629.0,한강초교보도육교 앞,용산구
132342113,2024-12-31 23:46:26,4629.0,한강초교보도육교 앞,용산구


In [47]:
df_final.to_csv('따릉이3개년이용내역.csv')

In [50]:
# '대여일시' 컬럼을 datetime 형식으로 변환
df_final['대여일시'] = pd.to_datetime(df_final['대여일시'], errors='coerce')

# 각각 연도, 월, 일, 시(hour) 컬럼으로 분해
df_final['연도'] = df_final['대여일시'].dt.year
df_final['월'] = df_final['대여일시'].dt.month
df_final['일'] = df_final['대여일시'].dt.day
df_final['시간'] = df_final['대여일시'].dt.hour

In [52]:
df_final.drop('대여일시',axis=1,inplace=True)

In [5]:
# '대여일시' 컬럼을 datetime 형식으로 변환
df_total['대여일시'] = pd.to_datetime(df_total['대여일시'], errors='coerce')

# 각각 연도, 월, 일, 시(hour) 컬럼으로 분해
df_total['연도'] = df_total['대여일시'].dt.year
df_total['월'] = df_total['대여일시'].dt.month
df_total['일'] = df_total['대여일시'].dt.day
df_total['시간'] = df_total['대여일시'].dt.hour

In [ ]:
df_total.to_csv('따릉이3개년이용내역.csv')

In [7]:
df_total.drop('자전거번호',axis=1,inplace=True)

In [12]:
df_total['대여소번호'] = df_total['대여 대여소번호'].replace('\\N',0).astype(int)

In [13]:
df_total = df_total[(df_total['대여소번호']>=102)&(df_total['대여소번호']<=6178)]

In [17]:
df_total.reset_index(drop=True,inplace= True)

In [21]:
df_total.drop(['대여일시','대여 대여소번호'],axis=1,inplace=True)

C:\Users\wjdwh\AppData\Local\Temp\ipykernel_23256\2062503806.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total.drop(['대여일시','대여 대여소번호'],axis=1,inplace=True)


In [22]:
df_total

,연도,월,일,시간,대여소번호
0,2022.0,1.0,1.0,9.0,102
1,2022.0,1.0,1.0,9.0,102
2,2022.0,1.0,1.0,10.0,102
3,2022.0,1.0,1.0,11.0,102
4,2022.0,1.0,1.0,12.0,102
...,...,...,...,...,...
132342110,2024.0,12.0,31.0,23.0,4032
132342111,2024.0,12.0,31.0,21.0,3759
132342112,2024.0,12.0,31.0,23.0,4629
132342113,2024.0,12.0,31.0,23.0,4629


In [23]:
df_total.to_csv('따릉이3개년이용내역_연도월일시간대여소번호.csv')

,Unnamed: 0,대여일시,대여 대여소번호,연도,월,일,시간,대여소번호
0,0,2022-01-01 09:02:01,102,2022.0,1.0,1.0,9.0,102
1,1,2022-01-01 09:42:06,102,2022.0,1.0,1.0,9.0,102
2,2,2022-01-01 10:05:30,102,2022.0,1.0,1.0,10.0,102
3,3,2022-01-01 11:14:51,102,2022.0,1.0,1.0,11.0,102
4,4,2022-01-01 12:22:38,102,2022.0,1.0,1.0,12.0,102
...,...,...,...,...,...,...,...,...
132342110,132342110,2024-12-31 23:14:10,4032,2024.0,12.0,31.0,23.0,4032
132342111,132342111,2024-12-31 21:45:25,3759,2024.0,12.0,31.0,21.0,3759
132342112,132342112,2024-12-31 23:46:21,4629,2024.0,12.0,31.0,23.0,4629
132342113,132342113,2024-12-31 23:46:26,4629,2024.0,12.0,31.0,23.0,4629


In [2]:
import pandas as pd
df = pd.read_csv('data/따릉이3개년이용내역_연도월일시간대여소번호.csv')

,Unnamed: 0,연도,월,일,시간,대여소번호
0,0,2022.0,1.0,1.0,9.0,102
1,1,2022.0,1.0,1.0,9.0,102
2,2,2022.0,1.0,1.0,10.0,102
3,3,2022.0,1.0,1.0,11.0,102
4,4,2022.0,1.0,1.0,12.0,102


In [4]:
df.shape

(132342115, 6)

In [7]:
# df_2022 = df[df['연도']==2022.0]
# df_2022.to_csv('따릉이22년도.csv',index=False)
df_2022.shape

(41594549, 6)

In [8]:
# df_2023 = df[df['연도']==2023.0]
df_2023.to_csv('따릉이23년도.csv',index=False)
# df_2023.shape

In [9]:
df_2024 = df[df['연도']==2024.0]
df_2024.to_csv('따릉이24년도.csv',index=False)

In [ ]:
import pandas as pd
test = pd.read_csv('따릉이22년도.csv')
test.drop('Unnamed: 0',axis=1,inplace = True)
data_2022 = test.groupby(['월','일'],as_index =False)['대여소번호'].value_counts()

In [ ]:
data_2022 = test.groupby(['월','일'],as_index =False)['대여소번호'].value_counts()

In [47]:
data_2022.to_csv('model_1/2022년_일별_대여소별_대여량.csv')

In [48]:
import pandas as pd
test = pd.read_csv('따릉이23년도.csv')
test.drop('Unnamed: 0',axis=1,inplace = True)
data_2023 = test.groupby(['월','일'],as_index =False)['대여소번호'].value_counts()
data_2022.to_csv('model_1/2023년_일별_대여소별_대여량.csv')

In [49]:
import pandas as pd
test = pd.read_csv('따릉이24년도.csv')
test.drop('Unnamed: 0',axis=1,inplace = True)
data_2023 = test.groupby(['월','일'],as_index =False)['대여소번호'].value_counts()
data_2022.to_csv('model_1/2024년_일별_대여소별_대여량.csv')

In [16]:
df_2023

,Unnamed: 0,연도,월,일,시간,대여소번호
41594549,41594549,2023.0,1.0,1.0,0.0,1554
41594550,41594550,2023.0,1.0,1.0,0.0,1653
41594551,41594551,2023.0,1.0,1.0,0.0,2601
41594552,41594552,2023.0,1.0,1.0,0.0,1080
41594553,41594553,2023.0,1.0,1.0,0.0,1351
...,...,...,...,...,...,...
87981593,87981593,2023.0,12.0,31.0,2.0,2539
87981594,87981594,2023.0,12.0,31.0,18.0,130
87981595,87981595,2023.0,12.0,31.0,15.0,5768
87981596,87981596,2023.0,12.0,31.0,16.0,451


In [17]:
df_2023.drop('Unnamed: 0',axis=1,inplace = True)
df_23 = df_2023.groupby(['월','일'],as_index =False)['대여소번호'].value_counts()
df_23.to_csv('model_1/2023년_일별_대여소별_대여량.csv')

C:\Users\wjdwh\AppData\Local\Temp\ipykernel_25212\3098695648.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023.drop('Unnamed: 0',axis=1,inplace = True)


In [19]:
df_2024.drop('Unnamed: 0',axis=1,inplace = True)
df_24 = df_2024.groupby(['월','일'],as_index =False)['대여소번호'].value_counts()
df_24.to_csv('model_1/2024년_일별_대여소별_대여량.csv')

C:\Users\wjdwh\AppData\Local\Temp\ipykernel_25212\619778793.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2024.drop('Unnamed: 0',axis=1,inplace = True)
